In [30]:
import os 
import pandas as pd 
import shutil
from IPython.display import display, HTML, Image, Markdown
import json
import geopandas
import pandas as pd
import os
import pickle
from tqdm import tqdm
import csv
import pprint

In [31]:
features = pd.read_csv('crowdflow-fairness/data/NY/features_without_pop.csv')
population = pd.read_csv('crowdflow-fairness/data/NY/population_new_york.csv')    

In [32]:
features

,geoid,residential_landuse,commercial_landuse,industrial_landuse,retail_landuse,forest_landuse,road_residential,road_primary,road_other,transport_point,food_point,health_point,education_point,retail_point
0,36105951701,0.462758,0.000000,0.0,0.000000,6.653214,120.909412,0.000000,52.798644,32,16,0,1,8
1,36105951801,0.000000,0.000000,0.0,0.000000,0.000000,40.978957,1.936949,7.213389,27,2,0,2,3
2,36105951302,0.494757,0.000000,0.0,0.000000,0.815496,127.057128,0.000000,27.155204,6,0,0,0,3
3,36105951802,0.119169,0.000000,0.0,0.000000,0.000000,30.384088,2.436651,2.079460,17,0,0,1,4
4,36105950801,0.483205,0.000000,0.0,0.000000,0.058853,96.812541,0.000000,17.163770,30,4,0,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5406,36055011711,0.000000,0.049299,0.0,0.137035,0.017674,13.023575,0.823414,3.344659,0,7,1,0,11
5407,36041950500,0.041368,0.000000,0.0,0.000000,132.215611,178.602673,120.816168,117.454053,30,6,1,3,7
5408,36041950400,0.167546,0.000000,0.0,0.000000,274.397277,131.751480,55.800271,73.987892,18,0,0,1,0
5409,36041950300,0.000000,0.000000,0.0,0.000000,322.239459,113.153546,87.908096,39.045088,21,6,0,1,7


In [33]:
population

,Label,total_population,geoid
0,Census Tract 1; Albany County; New York,"2,073",36001000100
1,Census Tract 2.01; Albany County; New York,"3,125",36001000201
2,Census Tract 2.02; Albany County; New York,"2,598",36001000202
3,Census Tract 3.01; Albany County; New York,"3,190",36001000301
4,Census Tract 3.02; Albany County; New York,"3,496",36001000302
...,...,...,...
5406,Census Tract 1503.01; Yates County; New York,"2,854",36123150301
5407,Census Tract 1503.02; Yates County; New York,"2,296",36123150302
5408,Census Tract 1504; Yates County; New York,"3,836",36123150400
5409,Census Tract 1505.01; Yates County; New York,"1,815",36123150501


In [34]:
# add population data to the updated_features_df
population_dict = population.set_index('geoid')['total_population'].to_dict()

for geoid in population_dict:
    # convert values of "xxx,xxxx" to numbers
    population_dict[geoid] = int(population_dict[geoid].replace(',', ''))

missing_geoids = []
total_population_values = [] 

for _, row in features.iterrows():
    geoid = row['geoid']
    if geoid in population_dict:
        total_population_values.append(population_dict[geoid])
    else:
        total_population_values.append(0)
        missing_geoids.append(geoid) 

features['total_population'] = total_population_values

if missing_geoids:
    print("GEOIDs not found in population_new_york:", missing_geoids)
else:
    print("All GEOIDs' population data were found and added.")

features.to_csv('crowdflow-fairness/data/NY/features.csv', index=False)

All GEOIDs' population data were found and added.
